In [1]:
from pathlib import Path
import pickle
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from scipy import sparse
from scipy.stats import norm

# Чтение данных

Я не стал добавлять код для обработки данных в этот ноутбук, он представлен в `data_processing.py`. В нем выполняются практически такие же операции, как и в первых 2 заданиях, в результате генерируются и сохранаются на диск 2 словаря. Их ключи -- названия признаков или групп признаков в случае one-hot кодирования, значения -- 2-мерные массивы значений. Прочитаем обработанные данные с помощью `pickle`.

In [2]:
DATA_DIR = Path("../data/processed")

with open(DATA_DIR / "train.pkl", "rb") as fin:
    train_data = pickle.load(fin)
with open(DATA_DIR / "test.pkl", "rb") as fin:
    test_data = pickle.load(fin)

feature_names = set(train_data.keys())
assert all(feature in feature_names for feature in test_data.keys())
{name: train_data[name].shape for name in feature_names}

{'coeff_sum0': (12041815, 1),
 'hour': (12041815, 1),
 'coeff_sum1': (12041815, 1),
 'country_id': (12041815, 17),
 'zone_id': (12041815, 771),
 'weekday': (12041815, 1),
 'g0': (12041815, 1),
 'banner_id': (12041815, 1052),
 'g1': (12041815, 1),
 'clicks': (12041815, 1),
 'os_id': (12041815, 8),
 'banner_id1': (12041815, 1052)}

# Линейная модель

In [3]:
def construct_arrays(data: dict, use_banner_id1: bool = False
                     ) -> tuple[sparse.csr_array, sparse.csr_array]:
    features = ["weekday", "hour", "zone_id", "country_id", "os_id"]
    features.append("banner_id1" if use_banner_id1 else "banner_id")
    x = sparse.hstack([data[feature] for feature in features], format="csr")
    y = data["clicks"].ravel()
    return x, y

In [4]:
x_train, y_train = construct_arrays(train_data, False)
model = LogisticRegression(C=5.0, solver="liblinear")
model.fit(x_train, y_train)
train_loss = log_loss(y_train, model.predict_proba(x_train)[:, 1])
print(f"train loss: {train_loss}")

x_test, y_test = construct_arrays(test_data, False)
test_loss = log_loss(y_test, model.predict_proba(x_test)[:, 1])
print(f"test loss: {test_loss}")


train loss: 0.10216763969008638
test loss: 0.13370016832950768


Значения log loss получились примерно такими же, как и первом задании.

# Расчёт clipped IPS

Для расчёта $\pi_0$ нужно определить вероятность того, что случайная величина $\xi_0 \sim N(\mu_0, \sigma_0)$ окажется больше чем $\xi_1 \sim N(\mu_1, \sigma_1)$. Для этого можно ввести случайную величину $\eta = \xi_1 - \xi_0 \sim N(\mu_1 - \mu_0, \sqrt{\sigma_1^2 + \sigma_0^2})$. Тогда искомая веростность $P(\xi_0 > \xi_1) = P(\eta < 0) = \mathcal{F}_\eta(0)$.

In [5]:
def compute_pi(mu_0: np.ndarray, mu_1: np.ndarray,
               sigma_0: np.ndarray, sigma_1: np.ndarray,
               min_var: float = 1e-7) -> np.ndarray:
    var = np.maximum(np.sqrt(sigma_0**2 + sigma_1**2), min_var)
    return norm.cdf((mu_1 - mu_0) / var).ravel()

In [6]:
pi_0_train = compute_pi(train_data["coeff_sum0"], train_data["coeff_sum1"],
                        train_data["g0"], train_data["g1"], min_var=1e-7)
pi_0_test = compute_pi(test_data["coeff_sum0"], test_data["coeff_sum1"],
                       test_data["g0"], test_data["g1"])

Для $\pi_1$ нам понадобятся логиты предсказанных моделей вероятностей для выбранных и алтернативных баннеров.

In [7]:
coeff_sum0_train = model.predict_log_proba(x_train)[:, 1]

x_train1, _ = construct_arrays(train_data, use_banner_id1=True)
coeff_sum1_train = model.predict_log_proba(x_train1)[:, 1]

coeff_sum0_test = model.predict_log_proba(x_test)[:, 1]

x_test1, _ = construct_arrays(test_data, use_banner_id1=True)
coeff_sum1_test = model.predict_log_proba(x_test1)[:, 1]

In [8]:
pi_1_train = compute_pi(coeff_sum0_train, coeff_sum1_train,
                        train_data["g0"].ravel(), train_data["g1"].ravel())
pi_1_test = compute_pi(coeff_sum0_test, coeff_sum1_test,
                        test_data["g0"].ravel(), test_data["g1"].ravel())

In [9]:
def compute_cips(r: np.array, pi_0: np.ndarray, pi_1: np.ndarray,
                 lambda_coef: float = 10., eps: float = 1e-7):
    return np.mean(r * np.clip(pi_1 / np.maximum(pi_0, eps), None, lambda_coef))

In [10]:
compute_cips(y_train, pi_0_train, pi_1_train)

0.06499140517867957

In [11]:
compute_cips(y_test, pi_0_test, pi_1_test)

0.08643633061000076

Для сравнения рассчитаем среднее число кликов.

In [12]:
y_train.mean(), y_test.mean()

(0.02469146054809844, 0.036045013178339795)